# Conversations -> Blog posts

## Setup

In [9]:
%pip install -U -q google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [10]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import json
import os
import time

In [11]:
genai.configure(api_key=os.environ['GEMINI_API_KEY'])

In [12]:
def upload_to_gemini(path, mime_type=None):
    file = genai.upload_file(path, mime_type=mime_type)
    print(f"Uploaded file '{file.display_name}' as: {file.uri}")
    return file

def wait_for_files_active(files):
    print("Waiting for file processing...")
    for name in (file.name for file in files):
        file = genai.get_file(name)
        while file.state.name == "PROCESSING":
            print(".", end="", flush=True)
            time.sleep(10)
            file = genai.get_file(name)
        if file.state.name != "ACTIVE":
            raise Exception(f"File {file.name} failed to process")
    print("...all files ready")
    print()

## Create the model

In [13]:
# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  safety_settings={
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }
)

## Get blog post ideas

In [38]:
files = [
  upload_to_gemini("conversations_txt/3.txt", mime_type="text/plain"),
]

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)

Uploaded file '3.txt' as: https://generativelanguage.googleapis.com/v1beta/files/94wxp8le2m8r
Waiting for file processing...
...all files ready



In [39]:


chat_session = model.start_chat(history=[])

response = chat_session.send_message([
    "Your task is to analyze the provided conversation and extract the 10 best blog post ideas. \nFor each idea, you should provide:\n1. A title\n2. A brief description (2-3 sentences)\n3. Two or more relevant quotes from the conversation\n\nFocus on topics that are interesting, thought-provoking and original",
    files[0]
])

print(response.text)

## 10 Blog Post Ideas from the Conversation:

**1. Title:** The Curious Case of the Self-Aware Poem: Exploring the Limits of Language Models

**Description:** This post analyzes a conversation between two students who use a powerful text generation AI to create poetry.  The AI’s output reveals a fascinating insight into its limitations and potential, particularly when it comes to understanding and replicating human nuances.  The conversation delves into the AI's ability to learn patterns and generate text, questioning whether it possesses true understanding or simply mimics human expression.

**Quotes:**

* "Now analyse the meaning of the shorter gap there"
* "err, the shorter gap indicates a stumble in its thinking, as if it were questioning what it was saying, or trying to reinforce it to itself, as if it were told to say it by a humn." 

**2. Title:**  Hacking Wordwall: How We Exploited a Website's Security Flaw

**Description:** This post details a humorous and educational tale of 

## Save

In [40]:
import time

# Save response text to a file
_id = time.time()
with open(f"ideas/{_id}.txt", "w", encoding="utf-8") as file:
    file.write(response.text)

print("Response saved to 'response_output.txt'")

Response saved to 'response_output.txt'


In [41]:
import json

# Prepare data for JSON
data = {
    "final_response": response.text,
    "chat_history": [
        {
            "role": entry.role,
            "parts": [str(part) for part in entry.parts]
        }
        for entry in chat_session.history
    ]
}

# Save data to a JSON file
with open(f"ideas/{_id}chat_history.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=2)

print("Response and chat history saved to 'chat_output.json'")

Response and chat history saved to 'chat_output.json'
